In [1]:
import os
import plotly

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import plotly.graph_objects as go
from tqdm import tqdm

In [2]:
def read_all_ticker_data(ticker, path):
    data = []
    for root, _, files in os.walk(path):
        for file in files:
            if file.startswith(ticker):
                df = pd.read_csv(os.path.join(root, file)).drop(
                    [
                        'Unnamed: 0', 
                        'TRADENO', 
                        'BOARDID',
                        'SECID', 
                        'PERIOD', 
                        'TRADETIME_GRP', 
                        'SYSTIME', 
                        'DECIMALS', 
                        'TRADINGSESSION',
                    ], 
                    axis=1,
                )
                df.TRADETIME = pd.to_datetime(
                    root.split("\\")[1] + ' ' + df.TRADETIME,
                )
                data.append(df)
    return data


In [3]:
tickers = [file_name.split('_')[0] for file_name in os.listdir(os.path.join('data', os.listdir('data')[0]))]

In [38]:
anom_list = []

for ticker in tqdm(tickers):
    data = read_all_ticker_data(ticker, 'data_2')
    df = pd.concat(data, ignore_index=True)

    # df['QUANTITY_BUY'] = 0
    # df['QUANTITY_SELL'] = 0
    # df.loc[df['BUYSELL'] == 'B', 'QUANTITY_BUY'] = df.loc[df['BUYSELL'] == 'B', 'QUANTITY']
    # df.loc[df['BUYSELL'] == 'S', 'QUANTITY_SELL'] = df.loc[df['BUYSELL'] == 'S', 'QUANTITY']
    df.set_index('TRADETIME', inplace=True)

    df_30min = df.resample('15T').sum(numeric_only=True)

    df_30min.columns = ['sum_' + str(col) for col in df_30min.columns.tolist()]

    df_30min.rename(columns={'sum_PRICE': 'mean_PRICE'}, inplace=True)
    df_30min['mean_PRICE'] = df['PRICE'].resample('15T').mean()
    df_30min['mean_PRICE'].fillna(method='ffill', inplace=True)
    
    max_value_in_24_hours = df_30min.iloc[::-1].mean_PRICE.rolling('4h').max()[::-1]

    df_30min['next_24hr_percentage_change'] = (
        (max_value_in_24_hours -
        df_30min['mean_PRICE']) / df_30min['mean_PRICE']
    ) * 100

    # Calculate the mean and standard deviation of the quantity
    rolling_mean_quantity = df_30min.rolling('24h')['sum_QUANTITY'].mean()
    rolling_std_quantity = df_30min.rolling('24h')['sum_QUANTITY'].std()

    # Calculate Z-scores
    df_30min['z_score'] = (df_30min['sum_QUANTITY'] -
                        rolling_mean_quantity) / rolling_std_quantity

    # Define a threshold for anomalies (e.g., Z-score greater than 3 or less than -3)
    threshold = 3

    # Identify anomalies
    anomalies = df_30min[abs(df_30min['z_score']) > threshold]

    fig = go.Figure(data=[go.Scatter(
        x=df_30min.index,
        y=df_30min['mean_PRICE'],
    )])

    anom = anomalies[anomalies.next_24hr_percentage_change ==
                    anomalies.next_24hr_percentage_change.max()]

    try:
        anom_list.append((ticker, anom['next_24hr_percentage_change'].values[0]))
    except:
        continue

100%|██████████| 240/240 [01:03<00:00,  3.79it/s]


In [39]:
sorted_anom_list = sorted(anom_list, key=lambda x: x[1], reverse=True)
sorted_anom_list[:10]

[('LSNG', 416.821737750078),
 ('NNSB', 249.80313070200714),
 ('LNZL', 192.70619036795094),
 ('YRSB', 161.40845070422534),
 ('VJGZ', 158.45383126494366),
 ('KCHE', 154.13363533408832),
 ('KTSB', 146.14570937458683),
 ('KZOS', 143.37640020482348),
 ('TGKB', 137.953216374269),
 ('KROT', 121.39685844637005)]

In [40]:
for ticker, _ in [('SBER', 1)]:  # sorted_anom_list[:5]:
    data = read_all_ticker_data(ticker, 'data')
    df = pd.concat(data, ignore_index=True)

    # df['QUANTITY_BUY'] = 0
    # df['QUANTITY_SELL'] = 0
    # df.loc[df['BUYSELL'] == 'B', 'QUANTITY_BUY'] = df.loc[df['BUYSELL'] == 'B', 'QUANTITY']
    # df.loc[df['BUYSELL'] == 'S', 'QUANTITY_SELL'] = df.loc[df['BUYSELL'] == 'S', 'QUANTITY']
    df.set_index('TRADETIME', inplace=True)

    df_30min = df.resample('15T').sum(numeric_only=True)

    df_30min.columns = ['sum_' + str(col) for col in df_30min.columns.tolist()]

    df_30min.rename(columns={'sum_PRICE': 'mean_PRICE'}, inplace=True)
    df_30min['mean_PRICE'] = df['PRICE'].resample('15T').mean()
    df_30min['mean_PRICE'].fillna(method='ffill', inplace=True)

    df_30min['next_24hr_percentage_change'] = (
        (df_30min['mean_PRICE'].shift(-24) -
        df_30min['mean_PRICE']) / df_30min['mean_PRICE']
    ) * 100

    # Calculate the mean and standard deviation of the quantity
    rolling_mean_quantity = df_30min.rolling('24h')['sum_QUANTITY'].mean()
    rolling_std_quantity = df_30min.rolling('24h')['sum_QUANTITY'].std()

    # Calculate Z-scores
    df_30min['z_score'] = (df_30min['sum_QUANTITY'] -
                        rolling_mean_quantity) / rolling_std_quantity

    # Define a threshold for anomalies (e.g., Z-score greater than 3 or less than -3)
    threshold = 3

    # Identify anomalies
    anomalies = df_30min[abs(df_30min['z_score']) > threshold]

    fig = go.Figure(data=[go.Scatter(
        x=df_30min.index,
        y=df_30min['mean_PRICE'],
    )])

    anom = anomalies[anomalies.next_24hr_percentage_change ==
                    anomalies.next_24hr_percentage_change.max()]


    # Add scatter for anomalies
    fig.add_trace(go.Scatter(
        x=anom.index, y=anom['mean_PRICE'],
        mode='markers',
        name='Anomalies',
        hoverinfo='x+y+text',
        text=['Next 24hr Change: {:.2f}%'.format(change) for change in anom['next_24hr_percentage_change']],
        marker=dict(color='red', size=10),
    ))

    # Customize the layout
    fig.update_layout(
        title='{} Candlestick Chart with Anomalies'.format(ticker),
        xaxis=dict(title='Date'),
        yaxis=dict(title='Price'),
    )

    # Show the plot
    fig.show()


In [7]:
n = 1000000
df[:n].PRICE.min(), df[:n].PRICE.max()


(8.62, 211.95)

In [8]:
df[:1000000]

,PRICE,QUANTITY,VALUE,BUYSELL
TRADETIME,,,,
2023-03-13 09:59:53,136.3,1,1363.0,B
2023-03-13 09:59:53,136.3,2,2726.0,B
2023-03-13 09:59:53,136.3,1,1363.0,S
2023-03-13 09:59:53,136.3,1,1363.0,S
2023-03-13 09:59:53,136.3,4,5452.0,S
...,...,...,...,...
2023-07-13 18:45:03,15.3,10,15300.0,S
2023-07-13 18:45:03,15.3,30,45900.0,S
2023-07-13 18:45:03,15.3,1,1530.0,S


In [9]:
anomalies[anomalies.next_24hr_percentage_change > 3].sum_QUANTITY.mean()


10969.7

In [10]:
anomalies[anomalies.next_24hr_percentage_change <= 3].sum_QUANTITY.mean()


5879.68

In [11]:
anomalies.z_score.max()

9.695896903357593

In [12]:
anomalies.sum_QUANTITY.max()


49788